# Урок 7. Задания.

In [2]:
import pandas as pd
import numpy as np

import requests
from io import BytesIO # для чтения данных в виде байтов

import telebot

### Задание 1

Загрузите [данные](https://docs.google.com/spreadsheets/d/e/2PACX-1vTrOh3gaTJXbujyd996nbU7QrYj-Q4Wd_AhjxbMzlSQLc3sD4fo7n9quD-yKqnqx2TbMspLjoocgN9R/pub?output=csv) из Google Sheets при помощи библиотеки requests. Посчитайте процент мужчин среди всех посетителей сайта, округлите ответ до целых.

    from io import BytesIO
    import requests

    req = requests.get(your_link)
    data = req.content
    df = pd.read_csv(BytesIO(data))

In [9]:
# Запишем в переменную url ссылку на данные

url = 'https://docs.google.com/spreadsheets/d/e/2PACX-1vTrOh3gaTJXbujyd996nbU7QrYj-Q4Wd_AhjxbMzlSQLc3sD4fo7n9quD-yKqnqx2TbMspLjoocgN9R/pub?output=csv'

In [10]:
# Запишем данные в переменную req

req = requests.get(url)
req

<Response [200]>

> 1 способ

In [5]:
data = req.content # байтовая строка
data

b'date,gender,users\r\n2020-06-14,Not specified,150\r\n2020-06-14,male,117\r\n2020-06-14,female,21'

In [6]:
df = pd.read_csv(BytesIO(data)) # BytesIO - для считывания байтовых данных 
df

,date,gender,users
0,2020-06-14,Not specified,150
1,2020-06-14,male,117
2,2020-06-14,female,21


>2 способ

In [11]:
# Разделим строку на элементы списка и пместим в переменную text

text = req.text.splitlines()
text

['date,gender,users',
 '2020-06-14,Not specified,150',
 '2020-06-14,male,117',
 '2020-06-14,female,21']

In [14]:
col_names = text[0] # названия колонок
content = text[1:] # остальные данные

In [23]:
# Разделим данные по запятой ('date,gender,users' -> ['date', 'gender', 'users'])

col_names = col_names.split(',')
content = [row.split(',') for row in content]

In [26]:
# Преобразуем данные в DataFrame

pd.DataFrame(content, columns = col_names)

,date,gender,users
0,2020-06-14,Not specified,150
1,2020-06-14,male,117
2,2020-06-14,female,21


>3 способ


In [28]:
pd.read_csv(url)

,date,gender,users
0,2020-06-14,Not specified,150
1,2020-06-14,male,117
2,2020-06-14,female,21


>

In [31]:
# Посчитаем количество мужчин среди всех посетителей сайта и запишем в переменную male_number

male_number = df.query('gender == "male"').users

In [32]:
# Посчитаем общее количество посетителей сайта и запишем в переменную users_number

users_number = df.users.sum()

In [33]:
# Найдем процент мужчин среди всех посетителей сайта и округлим ответ до целых

((male_number / users_number) * 100).round()

1    41.0
Name: users, dtype: float64

### Задание 2

Скачайте файл с данными посещаемости сайта Яндекс.Метрика (karpov_courses_test.csv). При помощи функции np.where создайте серию, в которую будет записываться 1, если посещения были сделаны роботами с браузера, содержащего Chrome в названии и 0, в других случаях. Поместите получившуюся серию в переменную chrome_robots

Hint: ответ должен быть в форме pd. Series

  Hint2: в питру более жёсткие требования к логическим действиям чем в пандасе - выражения должны быть окружены скобками и вместо or/and нужно писать | или &

  Может понадобиться:
  
   str.contains()


In [34]:
# Запишем путь к файлу в переменную path

path = '/Users/ola/Documents/Karpov/7. Знакомство с API/karpov_courses_test.csv'

In [35]:
# Считаем данные в переменную traffic_data

traffic_data = pd.read_csv(path)

In [36]:
traffic_data.head()

,date,browser,user_type,users
0,2020-06-14,Google Chrome,People,139
1,2020-06-14,Chrome Mobile,People,35
2,2020-06-14,Google Chrome,Robots,28
3,2020-06-14,Yandex Browser,People,22
4,2020-06-14,Firefox,People,15


In [38]:
# Запишем в переменную chrome_robots массив в котором будет 1,
# если посещения были сделаны роботами с браузера, содержащего Chrome в названии и 0, в других случаях. 

chrome_robots = np.where((traffic_data.user_type == 'Robots') & (traffic_data.browser.str.contains('Chrome')), 1, 0)
chrome_robots

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0,
       0, 0])

In [41]:
# Преобразуем полученный массив к типу pd.Series и перезапишем в переменную chrome_robots

chrome_robots = pd.Series(chrome_robots)
chrome_robots

0     0
1     0
2     1
3     0
4     0
5     0
6     0
7     0
8     0
9     0
10    0
11    0
12    0
13    0
14    0
15    0
16    0
17    0
18    0
19    1
20    0
21    0
22    0
23    0
dtype: int64

### Задание 3

У вас есть датасет sample_ads.csv с данными о кликах и показах по объявлению, в этом датасете есть пропуски. Вычислите количество строк без пропусков в колонке click_type. Запишите ответ в переменную not_nan_count

    ad_id - id объявления
    user_id - id пользователя
    click_type - тип клика (клик по заголовку объявления, клик по центру)

In [59]:
# Запишем путь к файлу в переменную path

path = '/Users/ola/Downloads/[SW.BAND] 7/[SW.BAND] Задания/sample_ads.csv'

In [60]:
# Считаем данные в переменную sample_ads

sample_ads = pd.read_csv(path)
sample_ads.head()

,ad_id,user_id,click_type
0,12345,qwert,top
1,12345,fsssc,center
2,12345,vfvgg,NaN
3,12345,asrvv,NaN
4,12345,tynuy,top


In [61]:
# 1 способ
# Запишем в переменную not_nan_count количество not null значений в колонке click_type

not_nan_count = sample_ads.query('click_type == click_type').shape[0]
not_nan_count

11

In [64]:
# 2 способ
# Запишем в переменную not_nan_count количество not null значений в колонке click_type

sample_ads.click_type.notna().sum()

11

### Задание 4
В списке lst находятся различные элементы. Создайте списк new_list и сделайте следующее:
- при помощи конструкции try except и цикла попробуйте преобразовать каждое значение в листе в int
- если это получилось - добавьте преобразованное значение  в список new_list


In [108]:
# Функция, преобразующая целые чисел в int и добавляющая только их в новый список

def convert_to_int(lst):
    new_list = []
    
    for element in lst:
        try:
            new_list.append(int(element))
        except:
            pass # заглушка
    return new_list

In [103]:
# Иная реализация такой функции (без создания нового списка)

def convert_to_int_without_newlist(lst):
    for i in range(len(lst)):
        try:
            lst[i] = int(lst[i])
        except:
            lst.pop(i)
    return lst 

In [104]:
lst = ['0', '2', 'Hello World', '11', 3, '3']

In [109]:
convert_to_int(lst)

[0, 2, 11, 3, 3]

In [107]:
convert_to_int_without_newlist(lst)

[0, 2, 11, 3, 3]

### Задание 5
А теперь попробуем воспользоваться знаниями использования Telegram API. Отправьте себе в личку любое послание, а в ответ к этому шагу напишите ник бота, от имени которого будете отправлять сообщения.

In [133]:
token = ''

In [134]:
bot = telebot.TeleBot(token)

In [135]:
chat_id = '610280374'
text = 'я твой сын'

In [136]:
bot.send_message(chat_id, text)

Никнейм бота - @test_olive_ola_bot